In [1]:
DATA_PATH_JSON = "./NLP-Teknofest24/NLPJSON/"
DATA_PATH_TXT = "./NLP-Teknofest24/NLPTXT-1028 Etiketlenen Raporlar/"

In [2]:
import json
d_list = list()
for i in range(1,5):
    with open(DATA_PATH_JSON+str(i)+"/all.jsonl",'r', encoding="utf-8") as f:
        for a in f.readlines():
            d_list.append(json.loads(a))

In [3]:
import pandas as pd
import numpy as np
import re
import random

In [4]:
meta_data = pd.DataFrame(d_list)

In [5]:
meta_data = meta_data.drop(["Comments","id"],axis=1)

In [6]:
meta_data

,text,label
0,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memen...,"[[32, 47, ANAT], [48, 59, ANAT], [60, 66, OBS-..."
1,BİLATERAL MAMOGRAFİ İNCELEMESİNDE; \nBilateral...,"[[36, 50, ANAT], [51, 65, ANAT], [66, 75, OBS-..."
2,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 45, ANAT], [46, 60, ANAT], [61, 71, OBS-..."
3,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[[31, 45, ANAT], [46, 70, ANAT], [71, 78, ANAT..."
4,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki meme ...,"[[32, 44, ANAT], [45, 59, ANAT], [60, 65, OBS-..."
...,...,...
1023,Unilateral Mammografi incelemesinde;\nSağ meme...,"[[37, 45, ANAT], [45, 55, ANAT], [56, 61, OBS-..."
1024,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSağ meme cilt...,"[[32, 40, ANAT], [41, 55, ANAT], [56, 65, OBS-..."
1025,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSağ meme cild...,"[[32, 40, ANAT], [41, 46, ANAT], [47, 54, OBS-..."
1026,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSol meme cilt...,"[[32, 40, ANAT], [41, 55, ANAT], [56, 62, OBS-..."


In [7]:
from sklearn.model_selection import train_test_split

In [9]:
# First split: Train+Validation and Test
train_val_df, test_df = train_test_split(meta_data, test_size=250, random_state=42)

# Second split: Train and Validation
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [10]:
train_df

,text,label
73,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 40, ANAT], [41, 45, ANAT], [46, 60, ANAT..."
541,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMeme parankimi...,"[[31, 45, ANAT], [46, 55, OBS-PRESENT], [61, 6..."
454,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memey...,"[[83, 95, ANAT], [96, 118, OBS-PRESENT], [120,..."
855,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 46, ANAT], [46, 60, ANAT], [61, 71, OBS-..."
61,BİLATERAL MAMOGRAFİ İNCELEMESİ\t\nBilateral me...,"[[32, 46, ANAT], [46, 61, ANAT], [62, 71, OBS-..."
...,...,...
177,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme p...,"[[31, 38, ANAT], [39, 43, ANAT], [44, 60, ANAT..."
529,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme c...,"[[31, 43, ANAT], [44, 58, ANAT], [59, 64, OBS-..."
396,BİLATERAL MAMOGRAFİ İNCELEMESİ \nMeme paterni ...,"[[32, 44, ANAT], [45, 57, OBS-PRESENT], [58, 7..."
680,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[[31, 45, ANAT], [46, 63, ANAT], [64, 73, ANAT..."


In [13]:
import re

def remove_redundant_spaces_regex(text: str) -> str:
  return re.sub(r"\s+", " ", text)

In [14]:
def transform_label_normal(text: str, label: list) -> list[dict]:
    label_list_dict = []
    start : int = label[0]
    end : int = label[1]
    catg : str = label[2]
    part = text[start:end]
    token_list = remove_redundant_spaces_regex(part.strip()).split(" ")
    for i, t in enumerate(token_list):
        new_repr_inner = {"start": start, "end": start+len(t), "label": catg, "token":t}
        start += len(t) + 1
        label_list_dict.append(new_repr_inner)
    return label_list_dict

In [15]:
def transform_df_labels(df: pd.DataFrame):
    for i in range(df.shape[0]):
        text : str = df["text"].iloc[i]
        label_list : list = df["label"].iloc[i]
        transformed_labels : list = []
        for label in label_list:
            transformed_labels.extend(transform_label_normal(text,label))
        df["label"].iloc[i] = transformed_labels
    return df

In [16]:
train_df_copy = train_df.copy()
train_df_copy = transform_df_labels(train_df_copy)
test_df_copy = test_df.copy()
test_df_copy = transform_df_labels(test_df_copy)
val_df_copy = val_df.copy()
val_df_copy = transform_df_labels(val_df_copy)
train_df_copy

,text,label
73,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[{'start': 31, 'end': 40, 'label': 'ANAT', 'to..."
541,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMeme parankimi...,"[{'start': 31, 'end': 35, 'label': 'ANAT', 'to..."
454,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memey...,"[{'start': 83, 'end': 86, 'label': 'ANAT', 'to..."
855,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[{'start': 31, 'end': 40, 'label': 'ANAT', 'to..."
61,BİLATERAL MAMOGRAFİ İNCELEMESİ\t\nBilateral me...,"[{'start': 32, 'end': 41, 'label': 'ANAT', 'to..."
...,...,...
177,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme p...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."
529,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme c...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."
396,BİLATERAL MAMOGRAFİ İNCELEMESİ \nMeme paterni ...,"[{'start': 32, 'end': 36, 'label': 'ANAT', 'to..."
680,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."


In [17]:
label_mapping = {
    'O': 0,
    'ANAT': 1,
    'OBS-PRESENT': 2,
    'OBS-UNCERTAIN':3,
    'OBS-ABSENT': 4,
    'IMPRESSION': 5,
}

In [18]:
import nltk
t = nltk.tokenize.NLTKWordTokenizer()
def transform_into_ner_tags(df: pd.DataFrame):
    for i in range(df.shape[0]):
        labels = df["label"].iloc[i]
        text = df["text"].iloc[i]
        count = 0
        ner_tags = []
        tokenized_text = remove_redundant_spaces_regex(text.replace("\xa0"," ").replace("\n"," ")).split()#t.tokenize(text)
        #if i==0:print(tokenized_text)
        for token in tokenized_text:
            #if i==0:print("token: ",token, "  label:",labels[count]["token"], "   count:",count)
            if count < len(labels) and (labels[count]["token"] in token or labels[count]["token"] == token or token in labels[count]["token"]):
                ner_tags.append(label_mapping[labels[count]["label"]])
                count+=1
            else: ner_tags.append(label_mapping["O"])
        df.iloc[i] = tokenized_text, ner_tags
    df.columns = ["tokens","ner_tags"]

    return df

In [19]:
train_df_copy = transform_into_ner_tags(train_df_copy)
test_df_copy = transform_into_ner_tags(test_df_copy)
val_df_copy = transform_into_ner_tags(val_df_copy)

In [21]:
def remove_impression_tags(l : list) -> list:
    for i in range(len(l)):
        if l[i] == 5:
            l[i] = 0
    return l


In [22]:
train_df_copy["ner_tags"].apply(remove_impression_tags)
test_df_copy["ner_tags"].apply(remove_impression_tags)
val_df_copy["ner_tags"].apply(remove_impression_tags)

919    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
159    [0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
607    [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, ...
674    [0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 1, 1, 2, 2, 2, ...
749    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, ...
                             ...                        
728    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 0, 1, 1, ...
540    [0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
274    [0, 0, 0, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, ...
521    [0, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 2, ...
711    [0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, ...
Name: ner_tags, Length: 195, dtype: object

In [26]:
type(train_df_copy["ner_tags"].iloc[0][0])

int

In [43]:
out_list = []
obs_p_list = []
obs_a_list = []
obs_u_list = []
anat_list = []
for i in range(train_df_copy.shape[0]):
    item = train_df_copy.iloc[i]
    txt = item["tokens"]
    lbl = item["ner_tags"]
    for ind,l in enumerate(lbl):
        a = txt[ind]
        if l == 2:
            obs_p_list.append(a)
        elif l == 3:
            obs_u_list.append(a)
        elif l == 4:
            obs_a_list.append(a)
        elif l == 1:
            anat_list.append(a)
        elif l == 0:
            out_list.append(a)

In [44]:
obsp_df = pd.DataFrame(pd.DataFrame(obs_p_list).value_counts().reset_index())
obsa_df = pd.DataFrame(pd.DataFrame(obs_a_list).value_counts().reset_index())
obsu_df = pd.DataFrame(pd.DataFrame(obs_u_list).value_counts().reset_index())
anat_df = pd.DataFrame(pd.DataFrame(anat_list).value_counts().reset_index())
out_df = pd.DataFrame(pd.DataFrame(out_list).value_counts().reset_index())

In [45]:
obsp_df.columns = ["words","count"]
obsu_df.columns = ["words","count"]
obsa_df.columns = ["words","count"]
anat_df.columns = ["words","count"]
out_df.columns = ["words","count"]

In [47]:
all_tags_list = []
all_tags_list.extend(obs_p_list)
all_tags_list.extend(obs_u_list)
all_tags_list.extend(obs_a_list)
all_tags_list.extend(anat_list)
all_tags_list.extend(out_list)

In [48]:
len(all_tags_list)

40263

In [63]:
hardcoded_dict = dict()
for i in all_tags_list:
    total_count = all_tags_list.count(i)
    if i not in hardcoded_dict and total_count >= 20:
        OC = out_list.count(i)/total_count
        ANC = anat_list.count(i)/total_count
        PC = obs_p_list.count(i)/total_count
        UC = obs_u_list.count(i)/total_count
        AC = obs_a_list.count(i)/total_count
        
        hardcoded_dict[i] = [OC,ANC,PC,UC,AC]
        


In [64]:
hardcoded_probs_df = pd.DataFrame(hardcoded_dict).T.reset_index()
hardcoded_probs_df.columns=["token","O","ANAT","OBS-PRESENT","OBS-UNCERTAIN","OBS-ABSENT"]

In [66]:
hardcoded_probs_df.to_csv("hardcoded_probs_df.csv")